Visualization #4: Top Fast Food Brand Presence by Geographies (Python)

In [54]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
from folium.plugins import FastMarkerCluster
import re
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 500)

In [55]:
#Import the dataset and keep relevant columns only
data=pd.read_csv('Datafiniti_Fast_Food_Restaurants.csv')
data.drop(columns=['keys'],inplace=True)
data.head(5)

,id,dateAdded,dateUpdated,address,categories,city,country,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,American Restaurant and Fast Food Restaurant,Thibodaux,US,29.814697,-90.814742,SONIC Drive In,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
1,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,Fast Food Restaurants,Thibodaux,US,29.814697,-90.814742,SONIC Drive In,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
2,AVwcopQoByjofQCxgfVa,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,Fast Food Restaurant,Pigeon Forge,US,35.803788,-83.580553,Taco Bell,37863,TN,https://www.yellowpages.com/pigeon-forge-tn/mi...,"http://www.tacobell.com,https://locations.taco..."
3,AVweXN5RByjofQCxxilK,2017-01-03T07:46:11Z,2018-06-26T02:59:51Z,3652 Parkway,Fast Food,Pigeon Forge,US,35.782339,-83.551408,Arby's,37863,TN,http://www.yellowbook.com/profile/arbys_163389...,"http://www.arbys.com,https://locations.arbys.c..."
4,AWQ6MUvo3-Khe5l_j3SG,2018-06-26T02:59:43Z,2018-06-26T02:59:43Z,2118 Mt Zion Parkway,Fast Food Restaurant,Morrow,US,33.562738,-84.321143,Steak 'n Shake,30260,GA,https://foursquare.com/v/steak-n-shake/4bcf77a...,http://www.steaknshake.com/locations/23851-ste...


In [56]:
most_similar_edited = [["Carl's Jr.", "Carl's Jr", 'Carls Jr'],
 ["McDonald's", "Mc Donald's", 'Mcdonalds', 'McDonalds'],
 ['Cook-Out', 'Cook Out', 'CookOut'],
 ["Steak 'n Shake",
  "STEAK 'N SHAKE",
  'Steak N Shake',
  'Steak n Shake',
  "Steak 'N Shake"],
 ['QDOBA Mexican Eats', 'Qdoba Mexican Eats'],
 ['Burger King', 'Burger King®'],
 ["Hardee's", 'Hardees'],
 ['Taco Time', 'TacoTime'],
 ["Arby's", 'Arbys'],
 ['Chick-fil-A', 'Chick-Fil-A', 'ChickfilA'],
 ['Subway', 'SUBWAY'],
 ['Kfc', 'KFC'],
 ["Jack's", 'Jacks'],
 ['Sonic Drive-In',
  'SONIC Drive-In',
  'SONIC Drive In',
  'Sonic DriveIn',
  'Sonic Drive-in'],
 ["Church's Chicken", 'Churchs Chicken'],
 ['Big Boys', 'Big Boy'],
 ['Dairy Queen', 'Dairy queen'],
 ['Guthries', "Guthrie's"],
 ['Chick-Fil-A', 'Chick-fil-A', 'ChickfilA'],
 ["Wendy's", 'Wendys'],
 ["Jimmy John's", 'Jimmy Johns'],
 ['Dairy Queen Grill Chill', 'Dairy Queen Grill & Chill'],
 ["Moe's Southwest Grill", 'Moes Southwest Grill'],
 ["Domino's Pizza", 'Dominos Pizza'],
 ["Rally's", 'Rallys'],
 ['Full Moon Bar-B-Que', 'Full Moon Bar B Que'],
 ["Guthrie's", 'Guthries'],
 ["McAlister's Deli", "Mcalister's Deli", 'McAlisters Deli'],
 ["Jason's Deli", 'Jasons Deli'],
 ['KFC', 'Kfc', 'KFC Kentucky Fried Chicken', 'KFC - Kentucky Fried Chicken'],
 ['Popeyes Louisiana Kitchen', "Popeye's Louisiana Kitchen"],
 ["Long John Silver's", 'Long John Silvers'],
 ['BLIMPIE', 'Blimpie'],
 ['Five Guys Burgers Fries', 'Five Guys Burgers & Fries'],
 ['SUBWAY', 'Subway'],
 ['Dairy Queen Grill & Chill', 'Dairy Queen Grill Chill'],
 ['Potbelly Sandwich Works', 'Pot Belly Sandwich Works'],
 ["Charley's Grilled Subs", 'Charleys Grilled Subs'],
 ["Jersey Mike's Subs", 'Jersey Mikes Subs'],
 ['In-N-Out Burger', 'InNOut Burger'],
 ["Culver's", "CULVER'S", 'Culvers'],
 ["Famous Dave's", 'Famous Daves'],
 ["Freddy's Frozen Custard Steakburgers",
  'Freddys Frozen Custard Steakburgers',
  "Freddy's Frozen Custard & Steakburgers"],
 ['Cook Out', 'Cook-Out', 'CookOut'],
 ['TacoTime', 'Taco Time'],
 ['Hooters', 'Roosters'],
 ['BurgerFi', 'Burgerfi'],
 ["Chen's Restaurant", "Chan's Restaurant"],
 ['Taco Del Mar', 'Taco del Mar'],
 ['SONIC Drive-In',
  'Sonic Drive-In',
  'SONIC Drive In',
  'Sonic DriveIn',
  'Sonic Drive-in'],
 ['Ciscos Taqueria', "Cisco's Taqueria"],
 ['China King', 'China Lin'],
 ["Bojangles' Famous Chicken 'n Biscuits",
  'Bojangles Famous Chicken n Biscuits'],
 ["Dominic's of New York", 'Dominics of New York'],
 ["Papa John's Pizza", 'Papa Johns Pizza'],
 ['Chanellos Pizza', 'Chanello’s Pizza'],
 ["Fazoli's", 'Fazolis'],
 ['Wing Street', 'Wingstreet'],
 ["George's Gyros Spot", "George's Gyros Spot 2"],
 ['Taco Johns', "Taco John's"],
 ['RUNZA', 'Runza'],
 ['Bru Burger Bar', 'Grub Burger Bar'],
 ["Taco John's", 'Taco Johns'],
 ["Bob's Burger Brew", "Bob's Burgers Brew", "Bob's Burgers Brew", "Bob's Burger Brew"],
 ['Best Burgers', 'Best Burger'],
 ['Burgermaster', 'Burger Master'],
 ["Dick's Drive-In", "DK's Drive-In"],
 ["Charley's Grill Spirits", "Charley's Grill & Spirits"],
 ['Tom Drive-in', "Tom's Drive-In"],
 ["Fox's Pizza Den", 'Foxs Pizza Den'],
 ["Mc Donald's", "McDonald's", 'Mcdonalds', 'McDonalds'],
 ['Taco CASA', 'Taco Casa'],
 ["Mcalister's Deli", "McAlister's Deli", 'McAlisters Deli'],
 ['Saras Too', "Sara's Too"],
 ['Backyard Burgers', 'Back Yard Burgers'],
 ["CULVER'S", "Culver's", 'Culvers'],
 ["Simple Simon's Pizza", 'Simple Simons Pizza'],
 ['China Sea', 'China Star', 'China Bear'],
 ["Dino's Drive In", "Dan's Drive In"],
 ["STEAK 'N SHAKE",
  "Steak 'n Shake",
  'Steak N Shake',
  'Steak n Shake',
  "Steak 'N Shake"],
 ['Stanfields Steak House', "Stanfield's Steakhouse"],
 ['Wingstreet', 'Wing Street'],
 ["Big Billy's Burger Joint", 'Big Billys Burger Joint'],
 ['Big Boy', 'Big Boys'],
 ["Frisch's Big Boy Restaurant", "1 Frisch's Big Boy Restaurant", 
  "40 Frisch's Big Boy Restaurant", "1 Frisch's Big Boy Restaurant",
  "90 Frisch's Big Boy Restaurant"],
 ['Fireplace Restaurant Lounge', 'Fireplace Restaurant & Lounge'],
 ["Carl's Jr", "Carl's Jr.", 'Carls Jr'],
 ["Rick's on the River", 'Ricks on the River'],
 ['Grub Burger Bar', 'Bru Burger Bar'],
 ["Franky's", "Grandy's"],
 ['Gyro X-Press', 'Gyro Express'],
 ['Dominos Pizza', "Domino's Pizza"],
 ["Pietro's Pizza Gallery of Games", "Pietro's Pizza & Gallery of Games"],
 ['Burrtio Amigos', 'Burrito Amigos'],
 ["Albee's Ny Gyros", "Albee's NY Gyros"],
 ['Gyro Stop', 'Gyro Spot'],
 ['Nicholas Restaurant', "Nicholas' Restaurant"],
 ['Mcdonalds', "McDonald's", "Mc Donald's", 'McDonalds'],
 ['Burgerfi', 'BurgerFi'],
 ["Ryan's", 'Ryans'],
 ['Taste of Buffalo Pizzeria', 'Taste Of Buffalo Pizzeria'],
 ['Bad Daddys Burger Bar', "Bad Daddy's Burger Bar"],
 ["Zaxby's", "Arby's"],
 ["Topper's Pizza", 'Toppers Pizza'],
 ['C J Drive In', 'C & J Drive In'],
 ['Full Moon Bar B Que', 'Full Moon Bar-B-Que'],
 ['China Lin', 'China King'],
 ["Raising Cane's Chicken Fingers", 'Raising Canes Chicken Fingers'],
 ["Mary's Pizza Shack", 'Marys Pizza Shack'],
 ['Peking Chinese Restaurants', 'Peking Chinese Restaurant'],
 ['Arbys', "Arby's"],
 ['SONIC Drive In',
  'Sonic Drive-In',
  'SONIC Drive-In',
  'Sonic DriveIn',
  'Sonic Drive-in'],
 ['Hardees', "Hardee's"],
 ['McDonalds', "McDonald's", "Mc Donald's", 'Mcdonalds'],
 ['Wendys', "Wendy's"],
 ['Papa Johns Pizza', "Papa John's Pizza"],
 ["George's Gyros Spot 2", "George's Gyros Spot"],
 ['ChickfilA', 'Chick-fil-A', 'Chick-Fil-A'],
 ['Rallys', "Rally's"],
 ['C & J Drive In', 'C J Drive In'],
 ['Steak N Shake',
  "Steak 'n Shake",
  "STEAK 'N SHAKE",
  'Steak n Shake',
  "Steak 'N Shake"],
 ["Popeye's Louisiana Kitchen", 'Popeyes Louisiana Kitchen'],
 ["DJ's Drive-In", "DK's Drive-In"],
 ["Dan's Drive In", "Dino's Drive In"],
 ['Best Burger', 'Best Burgers', 'Beef Burger'],
 ['Jimmy Johns', "Jimmy John's"],
 ['BaskinRobbins', 'Baskin-Robbins', 'Baskin Robbins'],
 ['Carls Jr', "Carl's Jr.", "Carl's Jr"],
 ['WG Grinders', 'Wg Grinders'],
 ['McAlisters Deli', "McAlister's Deli", "Mcalister's Deli"],
 ['Fazolis', "Fazoli's"],
 ['Marys Pizza Shack', "Mary's Pizza Shack"],
 ['Bojangles Famous Chicken n Biscuits',
  "Bojangles' Famous Chicken 'n Biscuits"],
 ['Jacks', "Jack's"],
 ["Hardee's/red Burrito", 'Hardees Red Burrito', "Hardee's/Red Burrito"],
 ['Captain Ds', "Captain D'S"],
 ['Mr Hero', 'Mr. Hero'],
 ["Chan's Restaurant", "Chen's Restaurant"],
 ['Ritters Frozen Custard', "Ritter's Frozen Custard"],
 ['Hot Dog on a Stick', 'Hot Dog On A Stick'],
 ['Jersey Mikes Subs', "Jersey Mike's Subs"],
 ['AW Restaurants',
  'Aw Restaurants',
  'AWRestaurants',
  'A W Restaurant',
  'AW Restaurant',
  'Jam Restaurants'],
 ['Long John Silvers', "Long John Silver's"],
 ["Rally's Hamburgers", 'Rallys Hamburgers'],
 ['HomeTown Buffet', 'Hometown Buffet'],
 ['Back Yard Burgers', 'Backyard Burgers'],
 ['Hardees Red Burrito', "Hardee's/red Burrito", "Hardee's/Red Burrito"],
 ["DK's Drive-In", "Dick's Drive-In", "DJ's Drive-In", "K's Drive In"],
 ['Baskin-Robbins', 'BaskinRobbins', 'Baskin Robbins'],
 ['Churchs Chicken', "Church's Chicken"],
 ['Blimpie', 'BLIMPIE'],
 ['Foxs Pizza Den', "Fox's Pizza Den"],
 ['Steak n Shake',
  "Steak 'n Shake",
  "STEAK 'N SHAKE",
  'Steak N Shake',
  "Steak 'N Shake"],
 ['Rallys Hamburgers', "Rally's Hamburgers"],
 ['Sonic DriveIn',
  'Sonic Drive-In',
  'SONIC Drive-In',
  'SONIC Drive In',
  'Sonic Drive-in'],
 ['Famous Daves', "Famous Dave's"],
 ['Beef Burger', 'Best Burger'],
 ['Dominics of New York', "Dominic's of New York"],
 ['Z-Pizza', 'zpizza'],
 ['KFC - Kentucky Fried Chicken', 'KFC Kentucky Fried Chicken'],
 ["Rockne's", 'Rocknes'],
 ["Hardee's/Red Burrito", "Hardee's/red Burrito", 'Hardees Red Burrito'],
 ['Aw Restaurants',
  'AW Restaurants',
  'AWRestaurants',
  'A W Restaurant',
  'AW Restaurant',
  'Jam Restaurants'],
 ['AWRestaurants', 'AW Restaurants', 'Aw Restaurants', 'AW Restaurant'],
 ["Hardee's Restaurant", "Hardee's Restaurants"],
 ["Hardee's Restaurants", "Hardee's Restaurant"],
 ["Stanfield's Steakhouse", 'Stanfields Steak House'],
 ['Dunkin Donuts', "Dunkin' Donuts"],
 ['Einstein Bros. Bagels', 'Einstein Bros Bagels'],
 ['Simple Simons Pizza', "Simple Simon's Pizza"],
 ['A W Restaurant', 'AW Restaurants', 'Aw Restaurants', 'AW Restaurant'],
 ['Einstein Bros Bagels', 'Einstein Bros. Bagels'],
 ['Roosters', 'Hooters'],
 ['Culvers', "Culver's", "CULVER'S"],
 ['Slice of Life', 'Slice Of Life'],
 ['Jasons Deli', "Jason's Deli"],
 ['Wg Grinders', 'WG Grinders'],
 ['Charleys Grilled Subs', "Charley's Grilled Subs"],
 ['Freddys Frozen Custard Steakburgers',
  "Freddy's Frozen Custard Steakburgers"],
 ['Moes Southwest Grill', "Moe's Southwest Grill"],
 ['CookOut', 'Cook-Out', 'Cook Out'],
 ['Peking Chinese Restaurant', 'Peking Chinese Restaurants'],
 ['InNOut Burger', 'In-N-Out Burger'],
 ["Nicholas' Restaurant", 'Nicholas Restaurant'],
 ['Chanello’s Pizza', 'Chanellos Pizza'],
 ['Ryans', "Ryan's"],
 ['Burger King®', 'Burger King'],
 ['Toppers Pizza', "Topper's Pizza"],
 ["Albee's NY Gyros", "Albee's Ny Gyros"],
 ['Qdoba Mexican Eats', 'QDOBA Mexican Eats'],
 ['Runza', 'RUNZA'],
 ['Slice Of Life', 'Slice of Life'],
 ['Mai-Tai Restaurant', 'Mai Tai Restaurant'],
 ['Gyro Express', 'Gyro X-Press'],
 ['zpizza', 'Z-Pizza'],
 ['Raising Canes Chicken Fingers', "Raising Cane's Chicken Fingers"],
 ['Rocknes', "Rockne's"],
 ['LL Hawaiian Barbecue', 'L L Hawaiian Barbecue', 'L L Hawaiian Barbeque'],
 ['Dairy queen', 'Dairy Queen'],
 ['Blakes Lotaburger', "Blake's Lotaburger"],
 ['Emidio & Sons Italian Restaurant', 'Emidio Sons Italian Restaurant'],
 ['Taste Of Buffalo Pizzeria', 'Taste of Buffalo Pizzeria'],
 ['L L Hawaiian Barbecue',
  'LL Hawaiian Barbecue',
  'L L Hawaiian Barbeque',
  'L & L Hawaiian Barbecue'],
 ['Killer Burgers', 'Killer Burger'],
 ["Steak 'N Shake",
  "Steak 'n Shake",
  "STEAK 'N SHAKE",
  'Steak N Shake',
  'Steak n Shake'],
 ['Burrito Amigos', 'Burrtio Amigos'],
 ["Zack's Hamburgers", "Jack's Hamburgers"],
 ['AW Restaurant',
  'AW Restaurants',
  'Aw Restaurants',
  'AWRestaurants',
  'A W Restaurant'],
 ['Jam Restaurants', 'AW Restaurants', 'Aw Restaurants'],
 ['Big Billys Burger Joint', "Big Billy's Burger Joint"],
 ['L L Hawaiian Barbeque', 'LL Hawaiian Barbecue', 'L L Hawaiian Barbecue'],
 ["Ritter's Frozen Custard", 'Ritters Frozen Custard'],
 ["Pietro's Pizza & Gallery of Games", "Pietro's Pizza Gallery of Games"],
 ["K's Drive In", "DK's Drive-In"],
 ['Killer Burger', 'Killer Burgers'],
 ["Dunkin' Donuts", 'Dunkin Donuts'],
 ['Farlows on the Water', "Farlow's On The Water"],
 ['Hometown Buffet', 'HomeTown Buffet'],
 ["Blake's Lotaburger", 'Blakes Lotaburger'],
 ["Jack's Hamburgers", "Zack's Hamburgers"],
 ["Cisco's Taqueria", 'Ciscos Taqueria'],
 ["Grandy's", "Franky's"],
 ["Farlow's On The Water", 'Farlows on the Water'],
 ["Bad Daddy's Burger Bar", 'Bad Daddys Burger Bar'],
 ['Baskin Robbins', 'BaskinRobbins', 'Baskin-Robbins'],
 ["Sara's Too", 'Saras Too'],
 ['T & L Hotdogs', 'T & L Hot Dogs'],
 ["Tom's Drive-In", 'Tom Drive-in'],
 ['Sonic Drive-in',
  'Sonic Drive-In',
  'SONIC Drive-In',
  'SONIC Drive In',
  'Sonic DriveIn'],
 ['Taco Casa', 'Taco CASA'],
 ['Emidio Sons Italian Restaurant', 'Emidio & Sons Italian Restaurant'],
 ['Fireplace Restaurant & Lounge', 'Fireplace Restaurant Lounge'],
 ['Mai Tai Restaurant', 'Mai-Tai Restaurant'],
 ['Ricks on the River', "Rick's on the River"],
 ['Taco del Mar', 'Taco Del Mar'],
 ['Five Guys Burgers & Fries', 'Five Guys Burgers Fries'],
 ['Mr. Hero', 'Mr Hero'],
 ["Captain D'S", 'Captain Ds'],
 ['Gyro Spot', 'Gyro Stop'],
 ["Charley's Grill & Spirits", "Charley's Grill Spirits"],
 ['Hot Dog On A Stick', 'Hot Dog on a Stick'],
 ['L & L Hawaiian Barbecue', 'L L Hawaiian Barbecue'],
 ['Pot Belly Sandwich Works', 'Potbelly Sandwich Works'],
 ['Burger Master', 'Burgermaster'],
 ["Freddy's Frozen Custard & Steakburgers",
  "Freddy's Frozen Custard Steakburgers"]]

In [57]:
def sortFirst(val): 
    return val[0]  

# sorts the array in ascending according to 1st element 
most_similar_edited.sort(key = sortFirst)  
most_similar_edited

[['A W Restaurant', 'AW Restaurants', 'Aw Restaurants', 'AW Restaurant'],
 ['AW Restaurant',
  'AW Restaurants',
  'Aw Restaurants',
  'AWRestaurants',
  'A W Restaurant'],
 ['AW Restaurants',
  'Aw Restaurants',
  'AWRestaurants',
  'A W Restaurant',
  'AW Restaurant',
  'Jam Restaurants'],
 ['AWRestaurants', 'AW Restaurants', 'Aw Restaurants', 'AW Restaurant'],
 ["Albee's NY Gyros", "Albee's Ny Gyros"],
 ["Albee's Ny Gyros", "Albee's NY Gyros"],
 ["Arby's", 'Arbys'],
 ['Arbys', "Arby's"],
 ['Aw Restaurants',
  'AW Restaurants',
  'AWRestaurants',
  'A W Restaurant',
  'AW Restaurant',
  'Jam Restaurants'],
 ['BLIMPIE', 'Blimpie'],
 ['Back Yard Burgers', 'Backyard Burgers'],
 ['Backyard Burgers', 'Back Yard Burgers'],
 ["Bad Daddy's Burger Bar", 'Bad Daddys Burger Bar'],
 ['Bad Daddys Burger Bar', "Bad Daddy's Burger Bar"],
 ['Baskin Robbins', 'BaskinRobbins', 'Baskin-Robbins'],
 ['Baskin-Robbins', 'BaskinRobbins', 'Baskin Robbins'],
 ['BaskinRobbins', 'Baskin-Robbins', 'Baskin Robbin

In [58]:
# let's create a dictionary to make name replace easier
match_name_dict = {}
for row in most_similar_edited:
    for similar_word in row:
        match_name_dict[similar_word] = row[0]
match_name_dict

{'A W Restaurant': 'Aw Restaurants',
 'AW Restaurants': 'Jam Restaurants',
 'Aw Restaurants': 'Jam Restaurants',
 'AW Restaurant': 'Aw Restaurants',
 'AWRestaurants': 'Aw Restaurants',
 'Jam Restaurants': 'Jam Restaurants',
 "Albee's NY Gyros": "Albee's Ny Gyros",
 "Albee's Ny Gyros": "Albee's Ny Gyros",
 "Arby's": "Zaxby's",
 'Arbys': 'Arbys',
 'BLIMPIE': 'Blimpie',
 'Blimpie': 'Blimpie',
 'Back Yard Burgers': 'Backyard Burgers',
 'Backyard Burgers': 'Backyard Burgers',
 "Bad Daddy's Burger Bar": 'Bad Daddys Burger Bar',
 'Bad Daddys Burger Bar': 'Bad Daddys Burger Bar',
 'Baskin Robbins': 'BaskinRobbins',
 'BaskinRobbins': 'BaskinRobbins',
 'Baskin-Robbins': 'BaskinRobbins',
 'Beef Burger': 'Best Burger',
 'Best Burger': 'Best Burgers',
 'Best Burgers': 'Best Burgers',
 "Big Billy's Burger Joint": 'Big Billys Burger Joint',
 'Big Billys Burger Joint': 'Big Billys Burger Joint',
 'Big Boy': 'Big Boys',
 'Big Boys': 'Big Boys',
 "Blake's Lotaburger": 'Blakes Lotaburger',
 'Blakes Lotab

In [59]:
#top fast food restaurant brands in the dataset
data['name']=data.name.str.replace('$','')
data['name']=data.name.str.replace("'",'')
data['name']=data.name.str.lower()    
data['name']=data.name.map(lambda x: re.sub(r'\W+','',x))


Brand_1=data.groupby(['name']).agg({'name':'count'})
Brand_1.columns=['number']
Brand_1=Brand_1.sort_values('number',ascending=False)

# Brand_1.head(10)
Brand_1.nlargest(15,'number').index

/var/folders/09/9q2rkflj5sg0ftxnxx2dpvb40000gn/T/ipykernel_80720/360620811.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



Index(['mcdonalds', 'tacobell', 'subway', 'burgerking', 'arbys', 'wendys',
       'jackinthebox', 'pizzahut', 'chickfila', 'dairyqueen', 'dominospizza',
       'sonicdrivein', 'kfc', 'carlsjr', 'jimmyjohns'],
      dtype='object', name='name')

In [60]:
#split the country into 4 key regions

def market(x):
    
    if x in ('WA','OR','CA','NV','AZ','NM','CO','WY','MT','ID','AK','HI'):
        return "West"
    elif x in ('ND','MN','IA','SD','KS','NE','MO','WI','MI','IL','IN','OH'):
        return "Mid West"
    elif x in ('NY','NJ','PA','CT','RI','MA','ME','NH','VT'):
        return "North East"
    else:
        return "South"

data['region']=data['province'].apply(market)
data.head(5)

,id,dateAdded,dateUpdated,address,categories,city,country,latitude,longitude,name,postalCode,province,sourceURLs,websites,region
0,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,American Restaurant and Fast Food Restaurant,Thibodaux,US,29.814697,-90.814742,sonicdrivein,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...,South
1,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,Fast Food Restaurants,Thibodaux,US,29.814697,-90.814742,sonicdrivein,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...,South
2,AVwcopQoByjofQCxgfVa,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,Fast Food Restaurant,Pigeon Forge,US,35.803788,-83.580553,tacobell,37863,TN,https://www.yellowpages.com/pigeon-forge-tn/mi...,"http://www.tacobell.com,https://locations.taco...",South
3,AVweXN5RByjofQCxxilK,2017-01-03T07:46:11Z,2018-06-26T02:59:51Z,3652 Parkway,Fast Food,Pigeon Forge,US,35.782339,-83.551408,arbys,37863,TN,http://www.yellowbook.com/profile/arbys_163389...,"http://www.arbys.com,https://locations.arbys.c...",South
4,AWQ6MUvo3-Khe5l_j3SG,2018-06-26T02:59:43Z,2018-06-26T02:59:43Z,2118 Mt Zion Parkway,Fast Food Restaurant,Morrow,US,33.562738,-84.321143,steaknshake,30260,GA,https://foursquare.com/v/steak-n-shake/4bcf77a...,http://www.steaknshake.com/locations/23851-ste...,South


In [61]:
#focus on top 7 brands only, McDonalds, Burger King, Taco Bell, Wendys, Arbys, KFC and Subway.
Brands=['mcdonalds', 'burgerking', 'tacobell', 'wendys', 'arbys', 'kfc','subway']
data1=data[data.name.isin(Brands)]

# print(data1.shape)
data1.head(10)

d1=data1.groupby(['region','name']).agg({'name':'count'})
d1.columns=['Stores']
d1.reset_index(inplace=True)
d1.sort_values(['region', 'Stores'], ascending=[True, False], inplace=True)
d1.reset_index(inplace=True)
d1.drop('index',inplace=True,axis=1)

import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(d1, x="Stores", y="name", color="region",
                 title="Top Fast Food Brands Presence by Region",
                 labels={"Stores":"Store Count"} 
                )

fig.update_layout(
    margin=dict(l=20, r=20, t=25, b=20)
#     paper_bgcolor="LightSteelBlue",
)
fig.show()

Visualization #5: Top Fast Food Categories from Top 500 Fast Food Restaurants Word Cloud

In [62]:
# check the data type of the columns
print("Data types:")
print(data.dtypes)

Data types:
id              object
dateAdded       object
dateUpdated     object
address         object
categories      object
city            object
country         object
latitude       float64
longitude      float64
name            object
postalCode      object
province        object
sourceURLs      object
websites        object
region          object
dtype: object


In [63]:
import matplotlib.pyplot as plt

In [64]:
#use the match_name_dict to replace names in the dataset to make it cleaner
names = data['name'].values
print("size:", len(names))

# replace names with their dictionary value
for i in range(len(names)):
    if match_name_dict.get(names[i]) != None:
        names[i] = match_name_dict[names[i]]

data['names'] = names

size: 10000


In [65]:
data['categories'].value_counts()

Fast Food Restaurant                                                                     3425
Fast Food Restaurants                                                                    3406
Fast Food                                                                                1777
Fast Food Restaurant and Burger Joint                                                     260
Fast food restaurants                                                                     191
Fast Food Restaurant and Mexican Restaurant                                               132
Fast Food Restaurant, American Restaurant, and Sandwich Place                             101
Fast Food Restaurant, Sandwich Place, and American Restaurant                              65
Sandwich Place and Fast Food Restaurant                                                    35
Fast Food Restaurant and American Restaurant                                               29
American Restaurant and Fast Food Restaurant                

In [66]:
# split the categories string data by ","
categories = data['categories'].values
for i in range(len(categories)):
    categories[i] = categories[i].split(",")

In [67]:
# update the "categories" column in the fastfood_data
data['categories'] = categories
data.head()

,id,dateAdded,dateUpdated,address,categories,city,country,latitude,longitude,name,postalCode,province,sourceURLs,websites,region,names
0,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,[American Restaurant and Fast Food Restaurant],Thibodaux,US,29.814697,-90.814742,sonicdrivein,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...,South,sonicdrivein
1,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,[Fast Food Restaurants],Thibodaux,US,29.814697,-90.814742,sonicdrivein,70301,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...,South,sonicdrivein
2,AVwcopQoByjofQCxgfVa,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,[Fast Food Restaurant],Pigeon Forge,US,35.803788,-83.580553,tacobell,37863,TN,https://www.yellowpages.com/pigeon-forge-tn/mi...,"http://www.tacobell.com,https://locations.taco...",South,tacobell
3,AVweXN5RByjofQCxxilK,2017-01-03T07:46:11Z,2018-06-26T02:59:51Z,3652 Parkway,[Fast Food],Pigeon Forge,US,35.782339,-83.551408,arbys,37863,TN,http://www.yellowbook.com/profile/arbys_163389...,"http://www.arbys.com,https://locations.arbys.c...",South,arbys
4,AWQ6MUvo3-Khe5l_j3SG,2018-06-26T02:59:43Z,2018-06-26T02:59:43Z,2118 Mt Zion Parkway,[Fast Food Restaurant],Morrow,US,33.562738,-84.321143,steaknshake,30260,GA,https://foursquare.com/v/steak-n-shake/4bcf77a...,http://www.steaknshake.com/locations/23851-ste...,South,steaknshake


In [135]:
# top 500 most appeared restaurants in the dataset and their corresponding category
top500_name_by_number = data['name'].value_counts()[:500].index.tolist()
top500_rest_categories = []
for name in top500_name_by_number:
    category = data[data.name == name]["categories"][:1].values[0]
    top500_rest_categories.append(category)

In [136]:
# remove stopword "Restaurants", "Restaurant", and "Fast Food Restaurants" in categories
stopwords = ["Fast Food Restaurants","Fast Food restaurants", "Fast Food", "Restaurants", "Restaurant", "restaurants", "restaurant"]
for i in range(len(top500_rest_categories)):
    for j in range(len(top500_rest_categories[i])):
        word = top500_rest_categories[i][j]
        # remove stopword as the only word given
        for stopword in stopwords:
            isRemoved = False
            if isRemoved == False and word == stopword:
                top500_rest_categories[i][j] = top500_rest_categories[i][j].replace(stopword, "")
                isRemoved = True

        # replace the stopword within a group of words
        for stopword in stopwords:
            if isRemoved == False and stopword in word:
                top500_rest_categories[i][j] = top50_rest_categories[i][j].replace(stopword, "")

In [137]:
top500_rest_categories

[[''],
 [' '],
 [' '],
 [' '],
 [''],
 [' '],
 [''],
 [''],
 [''],
 [' '],
 [''],
 ['American  and  '],
 [' '],
 [' '],
 [' '],
 [''],
 [' '],
 [''],
 [''],
 ['Burger Joint and   Gulf Breeze'],
 [' ', ' Mexican ', ' and Taco Place'],
 [''],
 [' '],
 [''],
 [''],
 [''],
 [''],
 [' '],
 [''],
 ['Diner', ' Breakfast Spot', ' and  '],
 [''],
 [' '],
 [' '],
 [''],
 [''],
 [' '],
 [''],
 [''],
 [''],
 ['Mexican  and  '],
 [''],
 [' '],
 [' '],
 [''],
 [' '],
 [''],
 ['Pizza Place', '  ', ' and Italian  Preston Park'],
 [''],
 [''],
 [' '],
 [''],
 [''],
 [' '],
 [''],
 [''],
 [''],
 [''],
 [' '],
 [''],
 ['American  and  '],
 [''],
 [''],
 [''],
 [' '],
 [''],
 [''],
 [' '],
 [''],
 [' '],
 [' '],
 [' '],
 [' '],
 [' '],
 [''],
 [''],
 [''],
 [' '],
 [''],
 [''],
 [''],
 [''],
 [''],
 ['Latin American  and  '],
 [' '],
 [''],
 [''],
 [''],
 ['Fried Chicken Joint and   Elmhurst'],
 [' '],
 [''],
 [''],
 ['Coffee & Tea / Breakfast / '],
 [''],
 [' '],
 [''],
 [''],
 [''],
 [''],
 [''],
 [' ']

In [138]:
from wordcloud import WordCloud

# empty string is declare 
text = "" 

# iterating through list of rows 
for row in top500_rest_categories : 
    # iterating through words in the row 
    for word in row:
        if len(word) == 0:
            continue
        # concatenate the words 
        if word[-1] == " ":
            word = word[:-1] # remove the last space character
        text = text + " " + word.replace(" ", "_") 
print("Vocabulary of our processed categories data:\n")
print(text)

Vocabulary of our processed categories data:

      American__and_     Burger_Joint_and___Gulf_Breeze  _Mexican _and_Taco_Place   Diner _Breakfast_Spot _and_    Mexican__and_    Pizza_Place _ _and_Italian__Preston_Park    American__and_         Latin_American__and_  Fried_Chicken_Joint_and___Elmhurst  Coffee_&_Tea_/_Breakfast_/         fast_food American _ _and_Sandwich_Place_Central_Omaha   _American _and_Burger_Joint Ice_Cream_Shop _ _and_Burger_Joint Sandwich_Place _Pizza_Place _and_     __and_American   __and_Japanese     American__and_  __and_Mexican Burger_Joint _ _and_American Fried_Chicken_Joint _ _and_Wings_Joint American__and_    Bakery_&_Pastries_/_American_/     _American _and_Fried_Chicken_Joint __Crown_Heights    _American _and_Seafood Sandwich_Place_and_     Burger_Joint _ _and_American          Greek__and_   American__and_ Mexican__and_  Mexican__and_  __and_Mexican       Pizza_Place _ _and_Italian  Fried_Chicken_Joint _American _and_    Hot_Dog_Joint_and_     Chinese__

In [139]:
import os
from wordcloud import WordCloud
import matplotlib.pyplot as plt

desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
font_path = os.path.join(desktop_path, "RobotoSlab-VariableFont_wght.ttf")

if os.path.isfile(font_path):
    print("Font file found.")
else:
    print("Font file not found.")

Font file found.


In [125]:
wordcloud = WordCloud(width=800, height=800, background_color='white',
                      min_font_size=10, font_path=font_path).generate(text)

plt.figure(figsize=(16, 16), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

ValueError: Only supported for TrueType fonts

From above wordcloud visualization, we can notice that most of the top 500 mentioned fast food restaurants has the categories of "take out", "American", and "Hamburger and Hot Dogs", which are the most unhealthy category. 